# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [4]:
client.*list*?


In [5]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [9]:
db = client.get_database("Ironhack")

In [10]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [11]:
db.list_collection_names()

['companies']

In [12]:
collection = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [74]:
proj1 = {"_id": 0,"name": 1}

In [75]:
list(collection.find({"name":'Babelgum'}, proj1))

[{'name': 'Babelgum'}]

In [ ]:
# Your Code


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [76]:
proj2 = {"_id": 0,"name": 1, "number_of_employees": 1}

In [77]:
list((collection.find({"number_of_employees": {"$gte": 5000}}, proj2).limit(20)).sort("number_of_employees",-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [78]:
proj3 = {"_id": 0,"name": 1, "founded_year": 1}

In [79]:
list(collection.find({"$and": [{"founded_year": {"$gt": 2000}}, {"founded_year": {"$lt":2005}}]}, proj3).limit(5))

[{'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Plaxo', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [107]:
proj4 = {"_id": 0,"name": 1, "ipo.valuation_amount": 1}

In [109]:
list(collection.find({"$and": [{"ipo.valuation_amount": {"$gt": 100000000}}, {"founded_year": {"$lt":2010}}]}, proj4).limit(5))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [82]:
proj5 = {"_id": 0,"name": 1, "number_of_employees": 1}

In [83]:
list((collection.find({"$and": [{"number_of_employees": {"$lt":1000}}, {"founded_year": {"$lt": 2005}}]}, proj5).limit(10)).sort("number_of_employees",-1))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Mozilla', 'number_of_employees': 800},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Yelp', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [56]:
list(collection.find({"partners": {"$exists":False}}).limit(3))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [87]:
proj7 = {"_id": 0, "name":1, "category_code":1}

In [88]:
list((collection.find({"category_code": None}, proj7)).limit(10))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [89]:
proj8 = {"_id": 0, "name":1, "number_of_employees":1}

In [95]:
list(collection.find({"$and": [{"number_of_employees": {"$gte": 100}}, {"number_of_employees": {"$lt":1000}}]}, proj8).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [105]:
proj9 = {"_id": 0,"name": 1, "ipo.valuation_amount": 1}

In [106]:
list((collection.find({}, proj9).limit(10)).sort("ipo.valuation_amount",-1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [125]:
proj10 = {"_id": 0,"name": 1, "number_of_employees": 1}

In [126]:
list((collection.find({}, proj10).limit(10)).sort("number_of_employees",-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [127]:
proj11 = {"_id": 0,"name": 1, "founded_month": 1}

In [128]:
list(collection.find({"founded_month": {"$gt": 6}}, proj11).limit(10))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [129]:
proj12 = {"_id": 0,"name": 1, "founded_year":1, "acquisition.price_amount": 1}

In [130]:
list(collection.find({"$and": [{"founded_year": {"$lt": 2000}}, {"acquisition.price_amount": {"$gt":100000}}]}, proj12).limit(10))

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld',
  'founded_year': 1999,
  'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets',
  'founded_year': 1999,
  'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000}},
 {'name': 'Zappos',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1000000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [137]:
proj13 = {"_id": 0,"name": 1, "acquisition": 1}

In [139]:
list((collection.find({"acquisition.acquired_year": {"$gt": 2010}}, proj13).sort("acquisition.price_amount",-1)).limit(5))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [133]:
proj14 = {"_id": 0,"name": 1, "founded_year": 1}

In [135]:
list((collection.find({}, proj14).limit(15)).sort("founded_year",-1))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'PeekYou', 'founded_year': 2012}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [140]:
proj15 = {"_id": 0,"name": 1, "founded_day": 1}

In [143]:
list(collection.find({"founded_day": {"$lte": 7}}, proj15).limit(10))

[{'name': 'Facebook', 'founded_day': 1},
 {'name': 'Omnidrive', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Geni', 'founded_day': 1},
 {'name': 'Fox Interactive Media', 'founded_day': 1},
 {'name': 'StumbleUpon', 'founded_day': 1},
 {'name': 'Gizmoz', 'founded_day': 1},
 {'name': 'Helio', 'founded_day': 1},
 {'name': 'eBay', 'founded_day': 1},
 {'name': 'Joost', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [147]:
proj16 = {"_id": 0,"name": 1, "category_code": 1, "number_of_employees": 1}

In [152]:
list(collection.find({"$and": [{"category_code": "web"}, {"number_of_employees": {"$gt":4000}}]}, proj16).sort("number_of_employees",-1))

[{'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Groupon', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [153]:
proj17 = {"_id": 0,"name": 1, "acquisition.price_amount": 1, "acquisition.price_currency_code": 1}

In [157]:
list(collection.find({"$and": [{"acquisition.price_amount": {"$gt":10000000}}, {"acquisition.price_currency_code": "EUR"}]}, proj17).limit(10))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [158]:
proj18 = {"_id": 0,"name": 1, "acquisition": 1}

In [160]:
list(collection.find({"acquisition.acquired_month": {"$lte": 3}}, proj18).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [163]:
proj19 = {"_id": 0,"name": 1, "founded_year": 1, "acquisition.acquired_year": 1}

In [166]:
list(collection.find({"$and": [{"founded_year": {"$gt": 2000}}, {"founded_year": {"$lt":2010}}, {"acquisition.acquired_year": {"$gt":2011}} ]}, proj19).limit(10))

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'blogTV',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Revision3',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'iContact',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Mashery',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Dailymotion',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'KickApps',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Netvibes',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [167]:
proj20 = {"_id": 0,"name": 1, "deadpooled_year": 1}

In [172]:
list((collection.find({"deadpooled_year": {"$ne":None}}, proj20)).limit(15).skip(3))

[{'name': 'Omnidrive', 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Sparter', 'deadpooled_year': 2008},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Mercora', 'deadpooled_year': 2008},
 {'name': 'Wesabe', 'deadpooled_year': 2010},
 {'name': 'Jangl SMS', 'deadpooled_year': 2008},
 {'name': 'Stickam', 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'deadpooled_year': 2008},
 {'name': 'EQO', 'deadpooled_year': 2012},
 {'name': 'AllofMP3', 'deadpooled_year': 2007},
 {'name': 'SellABand', 'deadpooled_year': 2010},
 {'name': 'Zlio', 'deadpooled_year': 2011},
 {'name': 'Jaiku', 'deadpooled_year': 2012},
 {'name': 'Spotplex', 'deadpooled_year': 2008}]